In [24]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import os 
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.utils import shuffle
from scipy import misc
import glob
from matplotlib.pyplot import imshow
from scipy.misc import imresize
import imageio
import time
import math
from numpy import genfromtxt
from array import array

# one hot coding
from sklearn.preprocessing import LabelBinarizer
from sklearn.neural_network import MLPClassifier

In [25]:
# Get ID
IDList_train = genfromtxt('/usr/home/hao/work_space/FP/ITCNets/OULP-C1V2_Pack/OULP-C1V2_SubjectIDList(FormatVersion1.0)/IDList_OULP-C1V2-A-{}_Gallery.csv'.format(85)
                        , delimiter=',', dtype=np.int32)
IDList_train = IDList_train[:,0]
#IDList_target = genfromtxt('/usr/home/hao/work_space/FP/ITCNets/OULP-C1V2_Pack/OULP-C1V2_SubjectIDList(FormatVersion1.0)/IDList_OULP-C1V2-A-{}_Probe.csv'.format(85)
#                        , delimiter=',', dtype=np.int32)

ID_label = IDList_train 
ID_label.shape



(3254,)

In [26]:
# Parameters for functions
n_class = 2254
batch_size = 80

In [27]:
# Get ID of the image as label
def image_ID(img_id):
    img_id = ''.join(map(str, img_id))
    if len(img_id) !=7:
        img_id = '0' *(7 - len(img_id)) + img_id
    return(np.array(img_id))

# For one subject

def get_subject(features_id):
    root_path = '/usr/home/hao/work_space/FP/ITCNets/OULP-C1V2_Pack'
    single_subject = []
    for ii in [1, 3, 5, 8, 10, 13, 15, 18, 20]:
        for start_frame in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
            new_feature = imread(os.path.join(root_path,'OULP-GEI-(88x128)-{}f_{}/Seq0{}'.format(ii, start_frame, 0), '{}/{}_85_GEI_{}f.png'.format(features_id, features_id, ii)))
            new_feature = imresize(new_feature, [64, 64], interp = 'nearest')
            single_subject.append(new_feature)
    #single_id = (np.ones(126) * ID)       
    return(single_subject)
            
            
#person = get_subject(features_id)
#print(np.array(person).shape)

In [28]:
# For n_class subjects
def get_data(ID_label, n_class):
    data_total = []
    id_total = []
    label = []
    for ID in ID_label[0:n_class]:
        features_id = image_ID(str(ID))
        
        single_id = (np.ones(126) * ID)
        single_id = single_id.reshape(126,1)
        #print(single_id.shape)
        
        #print(features_id)
        data_unit = get_subject(features_id)
        #data_total.append(data_unit)
        if len(data_total) < 100:
            data_total = data_unit
            id_total = single_id
        else:
            data_total = np.vstack((data_total, data_unit))
            id_total = np.vstack((id_total, single_id))
    data_total = np.reshape(data_total, [data_total.shape[0], data_total.shape[1], data_total.shape[2], 1] )
    data_total = np.array(data_total) / 255.0
    return(data_total, id_total)
        
#a, b = get_data(ID_label, n_class)
#print(np.array(a).shape)
#print(np.array(b).shape)
#a_ = a[0:80]
#print(a_.shape)

In [ ]:
x_train = []
x_val = []
y_train = []
y_val = []

# Split data
from sklearn.model_selection import train_test_split
[data_total, id_total] = get_data(ID_label, n_class)

In [21]:


# To ensure train data include all subjects
# 126 * 2254 = 284004
for i in range(0, 126 * n_class, 126):
    x_train_init = data_total[i:i+25]
    x_val_init = data_total[i+25:i+40]
    
    y_train_init = id_total[i:i+25]
    y_val_init = id_total[i+25:i+40]

# split the rest of the data
    ratio = 0.9
    x_train_rest, x_val_rest, y_train_rest, y_val_rest = train_test_split(data_total[i+40 : i+126], id_total[i+40 : i+126], train_size = ratio, random_state = 123)
    if len(x_train) < 2:
        x_train = np.vstack((x_train_rest,x_train_init))
        x_val = np.vstack((x_val_rest,x_val_init))
        y_train = np.vstack((y_train_rest,y_train_init))
        y_val = np.vstack((y_val_rest,y_val_init))
    else:
        x_train = np.vstack((x_train, x_train_rest,x_train_init))
        x_val = np.vstack((x_val, x_val_rest,x_val_init))
        y_train = np.vstack((y_train, y_train_rest,y_train_init))
        y_val = np.vstack((y_val, y_val_rest,y_val_init))
        if i%1260 == 0:
            print(i)
print(y_train.shape)
print(y_val.shape)
print(x_train.shape)
print(x_val.shape)

with open('/usr/home/hao/work_space/FP/ITCNets/Classifier/data/GEI64x64_For_train_x', 'wb') as f:
    np.save(f, x_train)
with open('/usr/home/hao/work_space/FP/ITCNets/Classifier/data/GEI64x64_For_train_y', 'wb') as f:
    np.save(f, x_val)
with open('/usr/home/hao/work_space/FP/ITCNets/Classifier/data/GEI64x64_For_val_x', 'wb') as f:
    np.save(f, y_train)
with open('/usr/home/hao/work_space/FP/ITCNets/Classifier/data/GEI64x64_For_val_y', 'wb') as f:
    np.save(f, y_val)
    

0
1260
2520
3780
5040
6300
7560
8820
10080
11340
(20400, 1)
(4800, 1)
(20400, 64, 64, 1)
(4800, 64, 64, 1)


#  Build GEINet

In [22]:
keep_prob = 0.6
# Input and label
inputs_ = tf.placeholder(tf.float32, [None, 64, 64, 1], name='inputs_')
target_ = tf.placeholder(tf.float32, [None, n_class], name='target_')

# Build layers of the GEINet

# 1. [64,64,1] to [64,64,18]
conv1 = tf.layers.conv2d(inputs_,filters = 18,kernel_size =[7,7], strides = 1, padding = 'same', activation=tf.nn.relu,
                        kernel_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
# pooling [64,64,18] to [32,32,9]
conv1 = tf.layers.max_pooling2d(conv1, (2,2), strides = 2, padding='same')

# Add normalization layer
conv1 = tf.layers.batch_normalization(conv1, training=True)
###############################################################################
# 2. [32,32,9] to [32,32,45]
conv2 = tf.layers.conv2d(conv1, 45, kernel_size =[5,5], strides = 1, padding = 'same', activation=tf.nn.relu,
                        kernel_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
# pooling [32,32,45] to [16,16,15]
conv2 = tf.layers.max_pooling2d(conv2, (3,3), strides = 2, padding='same')

# Add normalization layer
conv2 = tf.layers.batch_normalization(conv2, training=True)
###############################################################################
# Rebuild the output
shape = np.prod(conv2.get_shape().as_list()[1:])
conv2 = tf.reshape(conv2, [-1, shape])

# 1. FC [16,16,15] to [1*1024]
fc1 = tf.contrib.layers.fully_connected(conv2, 1024, activation_fn=tf.nn.relu)
fc1 = tf.nn.dropout(fc1, keep_prob)

# 2. FC [1*1024] to [1*512]
fc2  = tf.contrib.layers.fully_connected(fc1, 512, activation_fn=tf.nn.relu)

# Logits [1*512] to [1*n_class]
logits_ = tf.contrib.layers.fully_connected(fc2, n_class, activation_fn=None)
logits_ = tf.identity(logits_, name='logits_')

# cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_, labels=target_))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits_, 1), tf.argmax(target_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')*100

# Training

In [23]:
# Basic parameters
data_shape = x_train.shape[0]
val_shape = x_val.shape[0]
epochs = 800

save_model_path = '/usr/home/hao/work_space/FP/ITCNets/Classifier'
#count = 0

lb = LabelBinarizer().fit(np.array(range(n_class)))
train_ = []

train_id = []
val_ = []
val_id = []
iteration  = 0
val_batch_size = 80
n_batches = data_shape // batch_size

val_batches = val_shape // val_batch_size


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        print("The {}-th epochs start:".format(epoch +1 ))
        for ii in range(data_shape//batch_size -1):
            train_ = x_train[ii : ii + batch_size]
            train_id = y_train[ii : ii + batch_size]
      
                
            train_id = lb.transform(train_id)
           
      
            train_loss, _ = sess.run([cost, optimizer], 
                                     feed_dict = {inputs_: train_, 
                                                  target_: train_id})
            train_acc  = sess.run(accuracy, 
                                  feed_dict = {inputs_: train_, 
                                            target_: train_id})
            # For validation
            
            if(iteration%20 == 0):
             
                print('Batch {:>2}, Train Loss {:.4f}, Train Accuracy %{:.4f}'.format(iteration + 1,
                                                        train_loss,
                                                        train_acc))
            #if iteration == 20 or iteration%100 ==0:
                #validation_loss = []
                validation_acc = []
                
                
                for jj in range(val_shape // val_batch_size -1):
                    val_ = x_val[jj : jj + val_batch_size]
                    val_id = y_val[jj : jj + val_batch_size]
                    
                    val_id = lb.transform(val_id)
                    val_loss, _ = sess.run([cost, optimizer], 
                                     feed_dict = {inputs_: val_, 
                                                  target_: val_id})
                    val_acc = sess.run(accuracy, 
                                       feed_dict = {inputs_: val_, 
                                                    target_: val_id})
                    validation_acc.append(val_acc)
                mean_val_acc = np.mean(np.array(validation_acc))
                #print("Validation accuracy: %{:.3f},".format(mean_val_acc))
                print('Validation Accuracy %{:.4f}'.format(val_acc))
            iteration += 1                   
           
            
        #count += 1
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

The 1-th epochs start:
Batch  1, Train Loss 5.1095, Train Accuracy %100.0000
Validation Accuracy %100.0000
Batch 21, Train Loss 0.0000, Train Accuracy %100.0000
Validation Accuracy %98.7500
Batch 41, Train Loss 0.0000, Train Accuracy %100.0000
Validation Accuracy %98.7500
Batch 61, Train Loss 0.0001, Train Accuracy %100.0000
Validation Accuracy %100.0000
Batch 81, Train Loss 0.0000, Train Accuracy %98.7500
Validation Accuracy %100.0000
Batch 101, Train Loss 0.2754, Train Accuracy %98.7500
Validation Accuracy %100.0000
Batch 121, Train Loss 0.0000, Train Accuracy %100.0000
Validation Accuracy %100.0000
Batch 141, Train Loss 0.0000, Train Accuracy %100.0000
Validation Accuracy %100.0000


KeyboardInterrupt: 